# Azure AI Document Intelligence 기초
Azure AI Document Intelligence는 머신러닝 모델을 사용해서 문서로부터 키-값 쌍과 텍스트, 테이블을 추출하는 서비스다. 이 노트북에서는 [레이아웃 모델](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout) `prebuilt-layout`을 사용해서 PDF에서 데이터를 추출한다. 최신 API 버전에서는 마크다운 형식으로 출력하는 기능도 지원한다.

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout


# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- [Azure AI Document Intelligence 리소스](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource)의 엔드포인트와 키가 필요하다.
- [Python](https://www.python.org/downloads/release/python-31011/)(이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

In [ ]:

!pip install azure-identity==1.15.0
!pip install azure-ai-formrecognizer==3.3.2
!pip install jsonpickle

In [ ]:
import azure.ai.formrecognizer
print("azure.ai.formrecognizer", azure.ai.formrecognizer.__VERSION__)

## 라이브러리 및 환경변수 불러오기

In [ ]:
from azure.core.credentials import AzureKeyCredential

## 연동 설정

In [ ]:
# Azure AI Document Intelligence
document_intelligence_key: str = "<Your document intelligence key>"
document_intelligence_endpoint: str = "<Your document intelligence endpoint>"
document_intelligence_creds: str = AzureKeyCredential(document_intelligence_key)

# PDF 문서 구조 추출
PDF에 OCR을 사용하는 [레이아웃 모델](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout) `prebuilt-layout`은 고도화된 머신러닝 기반 문서 분석 API다. 이 API를 사용하면 다양한 형식의 문서를 구조화된 데이터로 반환받을 수 있다. 이 API는 마이크로소프트가 가진 강력한 광학문자인식(OCR) 기능과 딥러닝 모델을 결합하여 텍스트, 테이블, (체크 박스)선택 표시, 문서 구조를 추출한다.

In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
import jsonpickle

# formatting function
def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])

document_analysis_client = DocumentAnalysisClient(
    endpoint=document_intelligence_endpoint, credential=document_intelligence_creds
)

# URL로 분석하는 경우
# poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)

# sample document
filename = "../../data/최충헌.pdf"

with open(filename, "rb") as f:
    poller = document_analysis_client.begin_analyze_document("prebuilt-layout", document = f)

result = poller.result()

### Debug 용 코드

[jsonpickle](https://pypi.org/project/jsonpickle/) 라이브러리를 사용하면 `AnalyzeResult` 객체 구조와 일치하는 JSON을 저장할 수 있다. Document Intelligence의 분석 데이터로 이것저것 시도해보고 싶을 때 유용하다.

In [ ]:
# Debug 용(AnalyzeResult의 객체 구조를 유지하며 JSON으로 변환)
json_data = jsonpickle.encode(result)
with open('analyzed_data.json', "w", encoding='utf-8') as f:
    f.write(json_data)

# JSON으로부터 객체 구조 복원
# f = open("analyzed_data.json")
# json_str = f.read()
# result = jsonpickle.decode(json_str)

## 텍스트 줄의 필기 스타일
응답에는 각 텍스트 줄의 필기 스타일 여부와 신뢰도 점수가 들어있다.

In [ ]:
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

## 페이지
페이지 컬렉션은 서비스 응답에 있는 첫 번째 객체다. 레이아웃 모델은 인쇄 및 필기 스타일 텍스트를 `lines`와 `words`로 추출한다. 이 모델에서는 추출된 단어의 경계 다각형(`polygon`) 좌표와 `confidence`를 출력한다.

### 선택 표시
문서에서 선택 표시도 추출할 수 있다. 추출된 선택 표시는 각 페이지인 `pages` 컬렉션 안에 있다.
이 컬렉션 안에는 경계 다각형(`polygon`), `confidence`, `state`(`selected/unselected`)가 포함되어 있다. 연관 텍스트(추출된 경우)도 시작 인덱스(`offset`)와 `length`로 포함된다. `length`는 문서 텍스트 전체를 포함한 최상위 `content` 프로퍼티를 참조한다.

In [ ]:
for page in result.pages:
    print("----Analyzing layout from page #{}----".format(page.page_number))
    print(
        "Page has width: {} and height: {}, measured with unit: {}".format(
            page.width, page.height, page.unit
        )
    )

    for line_idx, line in enumerate(page.lines):
        words = line.get_words()
        print(
            "...Line # {} has word count {} and text '{}' within bounding polygon '{}'".format(
                line_idx,
                len(words),
                line.content,
                format_polygon(line.polygon),
            )
        )

        for word in words:
            print(
                "......Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' within bounding polygon '{}' and has a confidence of {}".format(
                selection_mark.state,
                format_polygon(selection_mark.polygon),
                selection_mark.confidence,
            )
        )

## 테이블
레이아웃 모델은 JSON으로 출력된 `pageResults` 섹션에 테이블을 추출한다. 추출된 테이블 정보에는 행과 열의 수 및 행과 열의 범위가 포함된다. 경계 다각형이 있는 각 셀은 셀의 영역이 `columnHeader`으로 인식됐는지 여부와 상관없이 정보와 함께 출력된다. 이 모델에서는 회전 테이블 추출을 지원한다. 각 테이블 셀에는 행과 열의 인덱스와 경계 다각형 좌표가 포함되어 있다. 셀 텍스트의 경우 모델은 시작 인덱스(`offset`)가 포함된 `span` 정보를 출력한다.

In [ ]:
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
            table_idx, table.row_count, table.column_count
        )
    )
    for region in table.bounding_regions:
        print(
            "Table # {} location on page: {} is {}".format(
                table_idx,
                region.page_number,
                format_polygon(region.polygon),
            )
        )
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content,
            )
        )
        for region in cell.bounding_regions:
            print(
                "...content on page {} is within bounding polygon '{}'".format(
                    region.page_number,
                    format_polygon(region.polygon),
                )
            )

## 단락
레이아웃 모델은 `analyzeResults`의 최상위 객체로 `paragraphs` 컬렉션에서 식별된 모든 텍스트 블록을 추출한다. 이 컬렉션의 각 항목은 텍스트 블록을 나타내며 추출된 텍스트(`content`)와 경계 다각형(`polygon`) 좌표를 포함한다. `span`은 문서의 텍스트 전체를 포함하는 최상위 `content` 프로퍼티의 텍스트 조각을 가리킨다.

In [ ]:
for paragraph_idx, paragraph in enumerate(result.paragraphs):
    print(
        "`Paragraph #{}: {}".format(
            paragraph_idx, paragraph.content
        )
    )